In [1]:
dataset_params = {
    'data_dir':'F:/ps1.3/Dataset',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': ['damper','insulator','plate','spacer','tower']
}
from super_gradients.training import models

yolo_nas_l = models.get("yolo_nas_l", pretrained_weights="coco")

best_model = models.get('yolo_nas_l',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="F:/ps1.3/YOLO_NAS_L/best_100.pth")

The console stream is logged into C:\Users\shoba\sg_logs\console.log


[2023-07-05 10:06:29] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-07-05 10:06:31] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-07-05 10:06:34] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-07-05 10:06:34] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-07-05 10:06:34] WARNING - export.py - Failed to import pytorch_quantization
[2023-07-05 10:06:34] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-07-05 10:06:34] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.
[2023-07-05 10:06:35] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gr

In [ ]:
import os

url = "Dataset-PS1-v2-1/test/images/DJI_0515_JPG.rf.bab1a28d12fef798a72c0eafb1c6ae2c.jpg"
predictions = best_model.predict(url, conf=0.6)
predictions2 = best_model.predict(url, conf=0.6)
# Extract the image name without the path and extension
image_name = os.path.splitext(os.path.basename(url))[0]

# Get Coordinates of Bounding Boxes
prediction_objects = list(predictions._images_prediction_lst)
print(prediction_objects)

In [ ]:
import numpy as np
class_names = ['damper', 'insulator', 'plate', 'spacer', 'tower']

prediction = prediction_objects[0]
image = np.array(prediction.image).astype(float)
bboxes = np.array(prediction.prediction.bboxes_xyxy).astype(float)
confidence = np.array(prediction.prediction.confidence).astype(float)
labels = np.array(prediction.prediction.labels).astype(int)
# print(image)


In [19]:
import os
import numpy as np

# Define the path to your validation images directory
val_images_dir = "Dataset-PS1-v2-1/valid/images"

# Get a list of all image filenames in the validation directory
image_files = os.listdir(val_images_dir)

# Initialize an empty list to store all predictions
allPredictions = []

# Iterate over each image file
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(val_images_dir, image_file)
    
    # Predict on the current image
    predictions = best_model.predict(image_path, conf=0.6)
    
    # Extract the image name without the path and extension
    image_name = os.path.splitext(os.path.basename(image_file))[0]
    
    # Iterate over each prediction object in the current image
    for prediction in predictions._images_prediction_lst:
        # Extract the features from the prediction object
        image = np.array(prediction.image).astype(float)
        bboxes = np.array(prediction.prediction.bboxes_xyxy).astype(float)
        confidence = np.array(prediction.prediction.confidence).astype(float)
        labels = np.array(prediction.prediction.labels).astype(int)
        
        # Create a dictionary to store the features
        features = {
            'image_name': image_name,
            'image': image,
            'bboxes': bboxes,
            'confidence': confidence,
            'labels': labels
        }
        
        # Append the features to the list of all predictions
        allPredictions.append(features)


In [23]:
print(allPredictions)

[{'image_name': 'DJI_0498_JPG.rf.6a2754d4a9630add0b5772c17c51921c', 'image': array([[[180., 176., 165.],
        [169., 165., 154.],
        [158., 154., 143.],
        ...,
        [143., 146., 151.],
        [125., 128., 133.],
        [116., 119., 124.]],

       [[172., 168., 157.],
        [160., 156., 144.],
        [160., 156., 145.],
        ...,
        [144., 147., 152.],
        [139., 142., 147.],
        [137., 140., 145.]],

       [[168., 166., 153.],
        [160., 158., 143.],
        [175., 173., 160.],
        ...,
        [142., 145., 150.],
        [130., 133., 138.],
        [119., 122., 127.]],

       ...,

       [[151., 146., 114.],
        [181., 176., 146.],
        [181., 176., 147.],
        ...,
        [ 81.,  71.,  72.],
        [ 84.,  74.,  75.],
        [ 85.,  75.,  76.]],

       [[119., 114.,  84.],
        [155., 150., 120.],
        [157., 152., 123.],
        ...,
        [100.,  90.,  91.],
        [108.,  98.,  99.],
        [112., 102., 103.

In [31]:
import os
import numpy as np
from sklearn.metrics import precision_recall_curve, auc

# Define the class names
class_names = ['damper', 'insulator', 'plate', 'spacer', 'tower']

# Initialize lists to store ground truth and prediction values
y_true = []
y_scores = []

# Iterate over the validation dataset and read the YOLO format labels
val_labels_dir = "Dataset/valid/labels"
for image_name in os.listdir(val_labels_dir):
    label_file = os.path.join(val_labels_dir, image_name)
    
    # Read the YOLO format labels from the file
    with open(label_file, 'r') as file:
        lines = file.readlines()
    
    # Extract the class labels and convert them to integer IDs
    labels = [int(line.split()[0]) for line in lines]
    
    # Assign 1 to the corresponding class label, 0 to others
    y_true_i = np.isin(np.arange(len(class_names)), labels).astype(int)
    y_true.append(y_true_i)
    
    # Add dummy prediction scores (assuming a confidence of 1.0 for simplicity)
    y_scores.append(np.ones_like(y_true_i))

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_scores = np.array(y_scores)

# Initialize a dictionary to store the precision-recall curves and mAP for each class
class_metrics = {}

# Calculate precision-recall curve and mAP for each class
for class_id, class_name in enumerate(class_names):
    y_true_class = y_true[:, class_id]
    y_scores_class = y_scores[:, class_id]
    
    # Calculate precision and recall
    precision, recall, _ = precision_recall_curve(y_true_class, y_scores_class)
    
    # Calculate average precision (AP) using the area under the precision-recall curve
    ap = auc(recall, precision)
    
    # Store precision, recall, and AP in the class_metrics dictionary
    class_metrics[class_name] = {
        'precision': precision,
        'recall': recall,
        'ap': ap
    }
    
    # Print mAP for the current class
    print(f"mAP@0.50 for {class_name}: {ap}")


mAP@0.50 for damper: 0.8333333333333333
mAP@0.50 for insulator: 0.9
mAP@0.50 for plate: 0.8
mAP@0.50 for spacer: 0.9
mAP@0.50 for tower: 0.9666666666666667
